In [2]:
import pandas as pd
import numpy as np
import os
import sys
import logging

# Configurar logging visual
logging.basicConfig(level=logging.INFO)

# --- CONFIGURACIÓN DE RUTAS ---
# 1. Directorio del Notebook
notebook_dir = os.getcwd()
# 2. Directorio Raíz (ProyectoIA)
project_root = os.path.dirname(notebook_dir)
# 3. Directorio de la App (ProyectoIA/app)
app_dir = os.path.join(project_root, 'app')

# Añadimos al PATH para poder importar
if project_root not in sys.path: sys.path.insert(0, project_root)
if app_dir not in sys.path: sys.path.insert(0, app_dir)


try:
    # Importamos la instancia ya creada en tu archivo
    from ml_service import ml_service
    # Importamos config para tener constantes útiles
    import app_config
    
    # Definimos nombres de columnas (necesarios para convertir arrays a dicts)
    FEATURE_NAMES = ['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate']
    
    print("✅ Servicio ML cargado correctamente.")
    
except ImportError as e:
    print(f"\n❌ Error de importación: {e}")
    raise e

2026/02/05 00:41:08 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/05 00:41:08 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/05 00:41:08 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/05 00:41:08 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/05 00:41:08 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/05 00:41:08 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/05 00:41:08 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/05 00:41:08 INFO mlflow.store.db.utils: Updating database tables
2026/02/05 00:41:08 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/05 00:41:08 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/05 00:41:09 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/05 00:41:09 INFO alembic.runtime

🔄 Searching for the most recent model in MLflow Model Registry...


c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Model loaded: model_v0502260017


✅ Scaler loaded.
✅ Servicio ML cargado correctamente.


In [3]:
print("🔄 Verificando estado del servicio...")

if ml_service.model is None:
    print("⚠️ El modelo no se cargó automáticamente. Intentando recargar...")
    ml_service.load_latest_model()
else:
    print("✅ El modelo está listo en memoria.")

if ml_service.scaler is None:
    print("⚠️ Advertencia: El Scaler no se ha cargado (posiblemente falta el run_id).")
else:
    print("✅ Scaler listo.")

🔄 Verificando estado del servicio...
✅ El modelo está listo en memoria.
✅ Scaler listo.


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

def predict_risk(age, sysbp, diabp, bs, bodytemp, heartrate):
    """
    Función adaptadora: Convierte los datos crudos al formato
    que espera ml_service.
    """
    # 1. Convertir lista de valores a Diccionario
    data_dict = {
        'Age': age,
        'SystolicBP': sysbp,
        'DiastolicBP': diabp,
        'BS': bs,
        'BodyTemp': bodytemp,
        'HeartRate': heartrate
    }
    
    # 2. Llamar al servicio
    class_idx, confidence = ml_service.predict(data_dict)
    
    # 3. Mapear resultado
    risk_labels = {0: "low risk", 1: "mid risk", 2: "high risk"}
    prediction_str = risk_labels.get(class_idx, "Unknown")
    
    return {
        "features": data_dict,
        "prediction": prediction_str,
        "confidence": confidence * 100,
        "class_id": class_idx
    }

# --- CASOS DE PRUEBA ---
test_cases = [
    {"name": "Paciente Bajo Riesgo", "data": [25, 110, 70, 6.5, 98.0, 72]},
    {"name": "Paciente Riesgo Medio", "data": [35, 135, 85, 10.0, 99.5, 80]},
    {"name": "Paciente Alto Riesgo", "data": [42, 160, 100, 14.0, 101.5, 95]}
]

# --- AQUÍ ESTABA EL FALTANTE ---
initial_results = []  # <--- Creamos la lista vacía

print("\n🧪 Ejecutando pruebas con MLService...\n")
for test in test_cases:
    result = predict_risk(*test['data'])
    
    # Guardamos el resultado en la lista para usarlo al final
    initial_results.append({
        "Patient": test['name'],
        "Pre_Prediction": result['prediction'],
        "Pre_Confidence": result['confidence']
    })
    
    print(f"📋 {test['name']}: {result['prediction']} ({result['confidence']:.2f}%)")


🧪 Ejecutando pruebas con MLService...

📋 Paciente Bajo Riesgo: low risk (69.62%)
📋 Paciente Riesgo Medio: high risk (95.78%)
📋 Paciente Alto Riesgo: high risk (99.95%)


In [12]:
print("\n🔄 MEDICAL FEEDBACK SIMULATION (Retraining)")
print("-" * 60)

feedback_case = {
    "data": [25, 110, 70, 6.5, 98.0, 72], 
    "doctor_label": "low risk" 
}
data_dict = dict(zip(FEATURE_NAMES, feedback_case['data']))

print(f"📍 Feedback case received: {feedback_case['doctor_label']}")
print("⚙️ Starting 'fine_tune' process within the service...")

try:
    new_run_id = ml_service.fine_tune(data_dict, feedback_case['doctor_label'])
    
    print("\n✅ Re-training successful!")
    print(f"🆔 New MLflow Run ID: {new_run_id}")
    print("🚀 In-memory model automatically updated.")
    
except Exception as e:
    print(f"❌ Error during fine-tuning: {e}")


🔄 MEDICAL FEEDBACK SIMULATION (Retraining)
------------------------------------------------------------
📍 Feedback case received: low risk
⚙️ Starting 'fine_tune' process within the service...
🧠 Retraining (Fine-Tuning) for: low risk
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


2026/02/05 00:53:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model registered in Model Registry: model_v0502260053
🚀 Model updated. Local accuracy: 1.00. Run ID: bee8a4db348c4881b083490d8b5b0f8b

✅ Re-training successful!
🆔 New MLflow Run ID: bee8a4db348c4881b083490d8b5b0f8b
🚀 In-memory model automatically updated.


Successfully registered model 'model_v0502260053'.
Created version '1' of model 'model_v0502260053'.


In [14]:
import pandas as pd

print("\n📊 FINAL VALIDATION: POST-TRAINING PREDICTIONS")
print("=" * 60)

comparison_data = []

for i, test in enumerate(test_cases):
    # Predict with the updated model
    new_result = predict_risk(*test['data'])
    old_result = initial_results[i]
    
    # Calculate confidence change
    diff = new_result['confidence'] - old_result['Pre_Confidence']
    
    print(f"👤 {test['name']}")
    print(f"   Before: {old_result['Pre_Prediction']} ({old_result['Pre_Confidence']:.2f}%)")
    print(f"   After:  {new_result['prediction']} ({new_result['confidence']:.2f}%)")
    print(f"   Change: {diff:+.2f}%")
    print("-" * 30)
    
    comparison_data.append({
        "Patient": test['name'],
        "Pred (Before)": old_result['Pre_Prediction'],
        "Conf (Before)": f"{old_result['Pre_Confidence']:.2f}%",
        "Pred (After)": new_result['prediction'],
        "Conf (After)": f"{new_result['confidence']:.2f}%",
        "Delta": f"{diff:+.2f}%"
    })

print("\n📉 COMPARISON TABLE")
df_results = pd.DataFrame(comparison_data)
display(df_results)


📊 FINAL VALIDATION: POST-TRAINING PREDICTIONS
👤 Paciente Bajo Riesgo
   Before: low risk (69.62%)
   After:  low risk (86.78%)
   Change: +17.17%
------------------------------
👤 Paciente Riesgo Medio
   Before: high risk (95.78%)
   After:  high risk (95.75%)
   Change: -0.03%
------------------------------
👤 Paciente Alto Riesgo
   Before: high risk (99.95%)
   After:  high risk (99.95%)
   Change: +0.00%
------------------------------

📉 COMPARISON TABLE


,Patient,Pred (Before),Conf (Before),Pred (After),Conf (After),Delta
0,Paciente Bajo Riesgo,low risk,69.62%,low risk,86.78%,+17.17%
1,Paciente Riesgo Medio,high risk,95.78%,high risk,95.75%,-0.03%
2,Paciente Alto Riesgo,high risk,99.95%,high risk,99.95%,+0.00%


## Conclusions

The deployed model demonstrates a robust capability for identifying High Risk maternal health cases with significant confidence, which is critical for early medical intervention. Furthermore, the implementation of the Continuous Learning pipeline (as seen in the cells above) allows the system to evolve. By integrating feedback loops—where doctors can verify or correct predictions—the model dynamically adjusts its weights without losing previous knowledge (Transfer Learning), ensuring it becomes more accurate over time as it is exposed to specific local demographics.